In [1]:
include("./trajopt/utils.jl")
include("./trajopt/dynamics.jl")
include("./funlopt/funl_utils.jl")
include("./funlopt/funl_dynamics.jl")
# include("./funlopt/funl_constraint.jl")
# include("./trajopt/scaling.jl")

diff_numeric (generic function with 1 method)

In [2]:
# load nominal trajectory
using JLD2, FileIO
@load "./data/quadstar_N15_traj" my_dict
xnom = my_dict["x"]
unom = my_dict["u"]
tnom = my_dict["t"];
N = size(xnom,2) - 1
dtnom = zeros(N)
for i in 1:N
    dtnom[i] = tnom[i+1]-tnom[i]
end

In [3]:
@load "./data/quadstar_N15_funl" my_dict
Qnom = my_dict["Q"]
Knom = my_dict["K"]
Znom = my_dict["Z"];

In [4]:
dynamics = QuadrotorDynamics()
ix = dynamics.ix
iu = dynamics.iu
decay_rate = 0.1
DLMI = NonlinearDLMI(decay_rate,ix,iu,dynamics.Cv,dynamics.Dvu)
is = DLMI.is

1

In [17]:
idx = 4
A,B = diff(dynamics,xnom[:,idx],unom[:,idx])

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -0.06405495660484975; 0.0 0.0 … -0.014635884276714924 0.0], [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 36.77822728944465 0.0; 0.0 0.0 0.0 16.521279407877348])

In [18]:
include("./funlopt/funl_dynamics.jl")

diff_numeric (generic function with 1 method)

In [19]:
idx = 4
Q = Qnom[:,:,idx]
# K = Ynom[:,:,idx] * inv(Qnom[:,:,idx])
K = Knom[:,:,idx]
Z = Znom[:,:,idx]
lam = 1.5
# Q = Q' + Q
X = vec(Q)
U = vcat(vec(K),vec(Z),lam)
Fx_n,Fu_n = diff_numeric(DLMI,X,U,A,B)
Fx,Fu = diff(DLMI,X,U,A,B)

([0.14626070958674095 0.24658811575935136 … 0.0 0.0; -0.31586172142965063 -1.6336695204988068 … 0.0 0.0; … ; 0.0 0.0 … -1.7916690259659984 0.07256587418590424; 0.0 0.0 … -0.03388436955363355 -1.6058226246603313], sparse([1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  135, 136, 137, 138, 139, 140, 141, 142, 143, 144], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 

In [20]:
sum(abs.(Fx_n-Fx)) / sum(abs.(Fx_n))

6.749765740040305e-12

In [21]:
sum(abs.(Fu_n-Fu)) / sum(abs.(Fx_n))

5.14559530345204e-12

In [23]:
Fx_n[1:6,1:6]

6×6 Matrix{Float64}:
  0.146261    0.246588   -0.354727   2.05125    0.0805884   -0.143641
 -0.315862   -1.63367     2.42202   -0.349944   0.449754     0.980759
  0.604421    3.22181    -4.58469    0.669639   1.05293     -0.876743
  2.91826    15.5555    -22.3772     3.28315    5.08375     -9.06128
  4.43595    23.6454    -34.0149     4.9146     7.77765    -13.7738
 -3.34158   -17.8119     25.6232    -3.70214   -5.82119     10.4257

In [24]:
Fx[1:6,1:6]

6×6 Matrix{Float64}:
  0.146261    0.246588   -0.354727   2.05125    0.0805884   -0.143641
 -0.315862   -1.63367     2.42202   -0.349944   0.449754     0.980759
  0.604421    3.22181    -4.58469    0.669639   1.05293     -0.876743
  2.91826    15.5555    -22.3772     3.28315    5.08375     -9.06128
  4.43595    23.6454    -34.0149     4.9146     7.77765    -13.7738
 -3.34158   -17.8119     25.6232    -3.70214   -5.82119     10.4257

In [25]:
i_upper = get_index_for_upper(ix)

78-element Vector{Int64}:
   1
  13
  14
  25
  26
  27
  37
  38
  39
  40
   ⋮
 136
 137
 138
 139
 140
 141
 142
 143
 144

In [27]:
Fx[i_upper,i_upper][1:6,1:6]

6×6 Matrix{Float64}:
  0.146261   0.0         0.0        0.0         0.0        0.0
 -0.315862   0.0731304   0.123294   0.0         0.0        0.0
  0.0       -0.631723   -3.26734    0.0         0.0        0.0
  0.604421   0.0         0.0        0.0731304   0.123294  -0.177363
  0.0        0.604421    3.22181   -0.315862   -1.63367    2.42202
  0.0        0.0         0.0        1.20884     6.44361   -9.16939

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark diff_numeric(DLMI,X,U,A,B)

In [ ]:
@benchmark diff(DLMI,X,U,A,B)

In [ ]:
Fu_n

In [ ]:
Fu

In [ ]:
DLMI.D

In [ ]:
Fu_n

In [ ]:
Fu

In [ ]:
print_jl(Fx)
print_jl(Fu)

# constraint

In [ ]:
idx = 4
# Q = Qnom[:,:,idx]
# K = Ynom[:,:,idx] * inv(Qnom[:,:,idx])
# Z = Znom[:,:,idx]
# lam = 1.3
Q = randn(3,3)
K = randn(2,3)
Z = randn(3,3)
lam = 1.5
X = vec(Q)
U = vcat(vec(K),vec(Z),lam)

In [ ]:
a = [1.5;2.0]
function evaluate(q,k)
    Q = reshape(q,(3,3))
    K = reshape(k,(2,3))
    return a'*K*Q*K'*a
end

In [ ]:
evaluate(vec(Q),vec(K))

In [ ]:
function eval_diff_numeric(q,k)
    ix = length(q)
    iu = length(k)

    eps_x = Diagonal{Float64}(I, ix)
    eps_u = Diagonal{Float64}(I, iu)
    fx = zeros(1,ix)
    fu = zeros(1,iu)

    h = 2^(-18)
    for i in 1:ix
        fx[:,i] .= (evaluate(q+h*eps_x[:,i],k) - evaluate(q-h*eps_x[:,i],k)) / (2*h)
    end
    for i in 1:iu
        fu[:,i] .= (evaluate(q,k+h*eps_u[:,i]) - evaluate(q,k-h*eps_u[:,i])) / (2*h)
    end
    return fx,fu
end

In [ ]:
Cm = DLMI.Cm

In [ ]:
fk = vec(K)' * kron(Q, a*a') + (kron(Q, a*a') * vec(K))'
# fk = 2 * vec(K)' * kron(Q, a*a')
fq = kron(a'*K,a'*K)

In [ ]:
fq_,fk_ = eval_diff_numeric(vec(Q),vec(K))

In [ ]:
println(sum(abs.(fq-fq_)))
println(sum(abs.(fk-fk_)))

In [ ]:
print_jl(fq_)
print_jl(fk_)